In [ ]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [ ]:
# Import tensorflow
import tensorflow as tf
import keras_cv
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

In [ ]:
# Import other libraries
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns

In [ ]:
def compute_CAM_convnext_xlarge(model, img):
    # Expand image dimensions to fit the model input shape
    img = np.expand_dims(img, axis=0)

    # Predict to get the winning class
    predictions = model.predict(img, verbose=0)
    label_index = np.argmax(predictions)

    # Get the weights of the final dense layer
    class_weights = model.layers[-1].get_weights()[0]
    class_weights_winner = class_weights[:, label_index]

    # Define the final convolutional layer of the ConvNetLarge model
    final_conv_layer = tf.keras.Model(
        model.get_layer('convnext_xlarge').input,
        model.get_layer('convnext_xlarge').get_layer('convnext_xlarge_stage_3_block_2_pointwise_conv_2').output
    )

    # Compute the convolutional outputs and resize to the original image size
    conv_outputs = final_conv_layer(img)
    resized_conv_outputs = tf.image.resize(conv_outputs, (96, 96))
    mat_for_mult = resized_conv_outputs.numpy()[0]

    print("Shapes before dot product:", mat_for_mult.shape, class_weights_winner.shape)

    # Compute the final output using the class weights

    final_output = np.dot(mat_for_mult.reshape((96*96, 2048)), class_weights_winner).reshape(96, 96)


    print("Shape after dot product:", final_output.shape)

    return final_output, label_index, predictions

In [ ]:
data_files = np.load('/kaggle/input/plants/public_data.npz', allow_pickle=True)
data = data_files['data']
data_gray = np.mean(data, axis=-1)
labels = data_files['labels']

In [ ]:
troll_indexes = [58, 95, 137, 138, 171, 207, 338, 412, 434, 486, 506, 529, 571, 599, 622, 658, 692, 701, 723, 725, 753, 779, 783, 827, 840, 880, 898, 901, 961, 971, 974, 989, 1028, 1044, 1064, 1065, 1101, 1149, 1172, 1190, 1191, 1265, 1268, 1280, 1333, 1384, 1443, 1466, 1483, 1528, 1541, 1554, 1594, 1609, 1630, 1651, 1690, 1697, 1752, 1757, 1759, 1806, 1828, 1866, 1903, 1938, 1939, 1977, 1981, 1988, 2022, 2081, 2090, 2150, 2191, 2192, 2198, 2261, 2311, 2328, 2348, 2380, 2426, 2435, 2451, 2453, 2487, 2496, 2515, 2564, 2581, 2593, 2596, 2663, 2665, 2675, 2676, 2727, 2734, 2736, 2755, 2779, 2796, 2800, 2830, 2831, 2839, 2864, 2866, 2889, 2913, 2929, 2937, 3033, 3049, 3055, 3086, 3105, 3108, 3144, 3155, 3286, 3376, 3410, 3436, 3451, 3488, 3490, 3572, 3583, 3666, 3688, 3700, 3740, 3770, 3800, 3801, 3802, 3806, 3811, 3821, 3835, 3862, 3885, 3896, 3899, 3904, 3927, 3931, 3946, 3950, 3964, 3988, 3989, 4049, 4055, 4097, 4100, 4118, 4144, 4150, 4282, 4310, 4314, 4316, 4368, 4411, 4475, 4476, 4503, 4507, 4557, 4605, 4618, 4694, 4719, 4735, 4740, 4766, 4779, 4837, 4848, 4857, 4860, 4883, 4897, 4903, 4907, 4927, 5048, 5080, 5082, 5121, 5143, 5165, 5171]
mask = np.ones(data.shape[0], bool)
mask[troll_indexes] = False
data = data[mask]
labels = labels[mask]

In [ ]:
X=data
y=labels

y[labels=="healthy"]=0
y[labels=="unhealthy"]=1

In [ ]:
# Convert labels to one-hot encoding format
y = tfk.utils.to_categorical(y,2)

# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=0.1, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=len(X_test), stratify=np.argmax(y_train_val,axis=1))

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
from tensorflow.keras.applications.convnext import preprocess_input
model = tfk.models.load_model('/kaggle/input/transferlearningar/TransferLearningAR/', compile=False)

In [ ]:
# Assuming X_train_convnet_large is your dataset for ConvNetLarge
num_images_to_plot = 5  # Change this according to the number of images you want to plot

for _ in range(num_images_to_plot):
    # Randomly select an image
    random_index_convnet_large = np.random.randint(0, len(X_train))
    random_image_convnet_large = X_train[random_index_convnet_large]

    # Compute CAM for the random image
    final_output_convnet_large, label_index_convnet_large, predictions_convnet_large = compute_CAM_convnext_xlarge(model, random_image_convnet_large)

    # Display the random image
    plt.figure(figsize=(12, 4))  # Adjust the figure size as needed
    plt.subplot(1, 2, 1)
    plt.imshow(random_image_convnet_large.astype('uint8'))
    plt.title('Random Image')
    plt.axis('off')

    # Display the CAM with colorbar
    plt.subplot(1, 2, 2)
    plt.imshow(final_output_convnet_large, cmap='plasma')
    plt.title(f'CAM for Label {label_index_convnet_large}')
    plt.colorbar()

    # Add a text annotation for the model's prediction
    prediction_text = f'Model Prediction: {np.argmax(predictions_convnet_large[0])}'
    plt.annotate(prediction_text, xy=(0.5, 0.02), xycoords='axes fraction', ha='center', fontsize=10, color='white')


    plt.show()


In [ ]:
# Predict labels for the entire test set
predictions = model.predict(X_test, verbose=0)

# Display the shape of the predictions
print("Predictions Shape:", predictions.shape)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Assuming that y_test and predictions are your true labels and predicted labels, respectively

# Replace class labels with "healthy" and "unhealthy"
class_labels = ['Healthy', 'Unhealthy']
y_test_labels = [class_labels[i] for i in np.argmax(y_test, axis=-1)]
predictions_labels = [class_labels[i] for i in np.argmax(predictions, axis=-1)]

# Compute the confusion matrix
cm = confusion_matrix(y_test_labels, predictions_labels, labels=class_labels)

# Compute the percentage values for each cell
cm_percent = cm / cm.sum(axis=1, keepdims=True) * 100

# Compute classification metrics
accuracy = accuracy_score(y_test_labels, predictions_labels)
precision = precision_score(y_test_labels, predictions_labels, average='macro', labels=class_labels)
recall = recall_score(y_test_labels, predictions_labels, average='macro', labels=class_labels)
f1 = f1_score(y_test_labels, predictions_labels, average='macro', labels=class_labels)

# Display the computed metrics
print('Accuracy:', accuracy.round(4))
print('Precision:', precision.round(4))
print('Recall:', recall.round(4))
print('F1:', f1.round(4))

# Plot the confusion matrix with percentage values
plt.figure(figsize=(10, 8))
sns.heatmap(cm_percent.T, annot=True, fmt=".2f", cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

